In [1]:
import sys

In [2]:
%load_ext autoreload

In [3]:
%autoreload 2

In [4]:
sys.path.append("../utils/")
sys.path.append("../")

In [13]:
import metrics
metrics_type='auc'

In [6]:
from agg_crossfit_results import *

In [7]:
pd.set_option('display.max_rows', 500)

In [17]:
def get_all_single_dirs(base_path, start_with):
    dir_dict = {}
    for folder in os.listdir(base_path):
        if folder.startswith(start_with):
            dir_dict[folder[:-20]] = os.path.join(base_path, folder, "1e-4/0/")
    return dir_dict



# Few shot results

In [11]:
def get_all_few_shot(dir_dict, fs_posfix, metrics_type='auc'):
    dfs = []
    for alg in dir_dict:
        print(alg)
        df1 =  make_few_shot_result_table(dir_dict[alg], postfix=fs_postfix, with_dash=True)
        df2 =  make_few_shot_result_table(dir_dict[alg], postfix=fs_postfix, with_dash=False)
        df = pd.concat([df1, df2])
        df['task_id'] = df['task_id'].astype(int)
        df['at_task'] = df['at_task'].astype(int)
        df['step'] = df['step'].astype(int)
        if df.empty:
            print("df", df)
            print(alg, with_dash, "hereeee")
            continue
        if "task_detail" in df.columns:
            df = pd.DataFrame(df.groupby(["task_name", "task_detail"])[metrics_type].max()).reset_index()
        else:
            df = pd.DataFrame(df.groupby(["task_name"])[metrics_type].max()).reset_index()
        df["alg"] = start_with
        dfs.append(df)
    all_fs_df = pd.concat(dfs)
    return all_fs_df

In [54]:
base_path = "../runs"
start_with = 'BiHNet_single'
fs_postfix = "_naive_16shot"

dir_dict_1 = get_all_single_dirs(base_path, start_with, fs_postfix)
all_fs_df_1 = get_all_few_shot(dir_dict_1, fs_posfix)

start_with = 'single'
dir_dict_2 = get_all_single_dirs(base_path, start_with, fs_postfix)
all_fs_df_2 = get_all_few_shot(dir_dict_2, fs_posfix)

all_fs_df = pd.concat([all_fs_df_1, all_fs_df_2])

In [56]:
latex_table = all_fs_df.sort_values(by=["task_name", "task_detail", "alg"]).to_latex(index=False)

In [57]:
print(latex_table)

\begin{tabular}{llrl}
\toprule
task_name & task_detail & auc & alg \\
\midrule
abusive & abusive & 0.848944 & BiHNet_single \\
abusive & hateful & 0.693093 & BiHNet_single \\
cad & affiliationdirectedabuse & 0.646838 & BiHNet_single \\
cad & identitydirectedabuse & 0.660469 & BiHNet_single \\
cad & persondirectedabuse & 0.571026 & BiHNet_single \\
dygen & hate & 0.526387 & BiHNet_single \\
ghc & hd & 0.627295 & BiHNet_single \\
ghc & vo & 0.792449 & BiHNet_single \\
hate & hateful & 0.765029 & BiHNet_single \\
hate & offensive & 0.882576 & BiHNet_single \\
jigsaw & identity_attack & 0.877680 & BiHNet_single \\
jigsaw & insult & 0.704102 & BiHNet_single \\
jigsaw & obscene & 0.873733 & BiHNet_single \\
jigsaw & threat & 0.848561 & BiHNet_single \\
jigsaw & toxicity & 0.668386 & BiHNet_single \\
personal_attack & a & 0.726092 & BiHNet_single \\
personal_attack & ra & 0.824863 & BiHNet_single \\
personal_attack & tpa & 0.800374 & BiHNet_single \\
ucc & antagonize & 0.555508 & BiHNet_singl

In [61]:
for _,group in all_fs_df.groupby(["task_name", "task_detail"]):
    print(pd.DataFrame(group))
    print("-----")

  task_name task_detail       auc            alg
0   abusive     abusive  0.848944  BiHNet_single
-----
  task_name task_detail       auc            alg
0   abusive     hateful  0.693093  BiHNet_single
-----
  task_name               task_detail       auc            alg
0       cad  affiliationdirectedabuse  0.646838  BiHNet_single
-----
  task_name            task_detail       auc            alg
0       cad  identitydirectedabuse  0.660469  BiHNet_single
-----
  task_name          task_detail       auc            alg
0       cad  persondirectedabuse  0.571026  BiHNet_single
-----
  task_name task_detail       auc            alg
0     dygen        hate  0.526387  BiHNet_single
-----
  task_name task_detail       auc            alg
0       ghc          hd  0.627295  BiHNet_single
-----
  task_name task_detail       auc            alg
0       ghc          vo  0.792449  BiHNet_single
-----
  task_name task_detail       auc            alg
0      hate     hateful  0.765029  BiHNet_single
--

In [62]:
all_fs_df.groupby("alg")[metrics_type].mean()

alg
BiHNet_single    0.710747
Name: auc, dtype: float64

## Instant Accuracy

In [15]:
def get_all_instant(dir_dict, metrics_type='auc'):
    dfs = []
    for alg in dir_dict:
        if alg =="pretrained" or 'Multitask' in alg:
            continue
        print(alg)
        df1 =  make_results_table(dir_dict[alg], metrics_type=metrics_type, with_dash=True)
        df2 =  make_results_table(dir_dict[alg], metrics_type=metrics_type, with_dash=False)
        df = pd.concat([df1, df2])
        df['task_id'] = df['task_id'].astype(int)
        df['at_task'] = df['at_task'].astype(int)
        df['step'] = df['step'].astype(int)
        df = df[df.task_id == df.at_task]
        if "task_detail" in  df.columns:
            # if df.groupby(["task_name", "task_detail"])["em"].transform(max) == 0.0:
            #     print("here")
            #     idx = df.groupby(["task_name", "task_detail"])["em"].transform(max)
            idx = df.groupby(["task_name", "task_detail"])[metrics_type].transform(max) == df[metrics_type]
        else:
            print("here")
            idx = df.groupby(["task_name"])["em"].transform(max) == df["em"]

        df = df[idx]                      
        df["alg"] = start_with
        dfs.append(df)
    all_instant_df = pd.concat(dfs)
    return all_instant_df
    

In [20]:
base_path = "../runs"
start_with = 'BiHNet_single'

dir_dict_1 = get_all_single_dirs(base_path, start_with)
all_instant_df_1 = get_all_instant(dir_dict_1)

start_with = 'single'
dir_dict_2 = get_all_single_dirs(base_path, start_with)
all_instant_df_2 = get_all_instant(dir_dict_2)

all_instant_df = pd.concat([all_fs_df_1, all_fs_df_2])

BiHNet_single_personal_attack-ra
results_task_(\d+)_(\w+)-(\w+)_task_(\d+)_step_([\d]+).csv
Index(['task_id', 'task_name', 'task_detail', 'at_task', 'step', 'path', 'auc',
       'maj'],
      dtype='object')
results_task_(\d+)_(\w+)_task_(\d+)_step_([\d]+).csv
BiHNet_single_ghc-hd
results_task_(\d+)_(\w+)-(\w+)_task_(\d+)_step_([\d]+).csv
Index(['task_id', 'task_name', 'task_detail', 'at_task', 'step', 'path', 'auc',
       'maj'],
      dtype='object')
results_task_(\d+)_(\w+)_task_(\d+)_step_([\d]+).csv
BiHNet_single_jigsaw-insult
results_task_(\d+)_(\w+)-(\w+)_task_(\d+)_step_([\d]+).csv
Index(['task_id', 'task_name', 'task_detail', 'at_task', 'step', 'path', 'auc',
       'maj'],
      dtype='object')
results_task_(\d+)_(\w+)_task_(\d+)_step_([\d]+).csv
BiHNet_single_jigsaw-identity_attack
results_task_(\d+)_(\w+)-(\w+)_task_(\d+)_step_([\d]+).csv
Index(['task_id', 'task_name', 'task_detail', 'at_task', 'step', 'path', 'auc',
       'maj'],
      dtype='object')
results_task_(\d+)

Index(['task_id', 'task_name', 'task_detail', 'at_task', 'step', 'path', 'auc',
       'maj'],
      dtype='object')
results_task_(\d+)_(\w+)_task_(\d+)_step_([\d]+).csv
single_ucc-dismissive
results_task_(\d+)_(\w+)-(\w+)_task_(\d+)_step_([\d]+).csv
Index(['task_id', 'task_name', 'task_detail', 'at_task', 'step', 'path', 'auc',
       'maj'],
      dtype='object')
results_task_(\d+)_(\w+)_task_(\d+)_step_([\d]+).csv
single_personal_attack-a
results_task_(\d+)_(\w+)-(\w+)_task_(\d+)_step_([\d]+).csv
Index(['task_id', 'task_name', 'task_detail', 'at_task', 'step', 'path', 'auc',
       'maj'],
      dtype='object')
results_task_(\d+)_(\w+)_task_(\d+)_step_([\d]+).csv
single_hate-offensive
results_task_(\d+)_(\w+)-(\w+)_task_(\d+)_step_([\d]+).csv
Index(['task_id', 'task_name', 'task_detail', 'at_task', 'step', 'path', 'auc',
       'maj'],
      dtype='object')
results_task_(\d+)_(\w+)_task_(\d+)_step_([\d]+).csv
single_jigsaw-toxicity
results_task_(\d+)_(\w+)-(\w+)_task_(\d+)_step_([\d

In [21]:
all_instant_df

,task_id,task_name,task_detail,at_task,step,path,auc,maj,alg
6,0,personal_attack,ra,0,1099,../runs/BiHNet_single_personal_attack-ra_0.01_...,0.966567,0.911269,BiHNet_single
3,0,ghc,hd,0,785,../runs/BiHNet_single_ghc-hd_0.01_s64_d256_lim...,0.857989,0.920145,BiHNet_single
2,0,jigsaw,insult,0,942,../runs/BiHNet_single_jigsaw-insult_0.01_s64_d...,0.946066,0.940900,BiHNet_single
0,0,jigsaw,identity_attack,0,1099,../runs/BiHNet_single_jigsaw-identity_attack_0...,0.983310,0.994100,BiHNet_single
3,0,abusive,hateful,0,628,../runs/BiHNet_single_abusive-hateful_0.01_s64...,0.853027,0.942422,BiHNet_single
5,0,ucc,antagonize,0,942,../runs/BiHNet_single_ucc-antagonize_0.01_s64_...,0.795998,0.960696,BiHNet_single
10,0,cad,affiliationdirectedabuse,0,1413,../runs/BiHNet_single_cad-affiliationdirecteda...,0.890791,0.945647,BiHNet_single
3,0,personal_attack,tpa,0,942,../runs/BiHNet_single_personal_attack-tpa_0.01...,0.897092,0.990371,BiHNet_single
2,0,ucc,hostile,0,628,../runs/BiHNet_single_ucc-hostile_0.01_s64_d25...,0.744352,0.977637,BiHNet_single
12,0,hate,offensive,0,1413,../runs/BiHNet_single_hate-offensive_0.01_s64_...,0.976745,0.876065,BiHNet_single


In [22]:
all_instant_df = all_instant_df[[metrics_type, "task_name", "task_detail",  "alg"]]
# all_instant_df= all_instant_df.fillna("singlelabel")
for _,group in all_instant_df.groupby(["task_name", "task_detail"]):
    print(pd.DataFrame(group))

        auc task_name task_detail            alg
3  0.972980   abusive     abusive  BiHNet_single
0  0.975115   abusive     abusive         single
        auc task_name task_detail            alg
3  0.853027   abusive     hateful  BiHNet_single
0  0.866319   abusive     hateful         single
         auc task_name               task_detail            alg
10  0.890791       cad  affiliationdirectedabuse  BiHNet_single
0   0.900591       cad  affiliationdirectedabuse         single
         auc task_name            task_detail            alg
10  0.790011       cad  identitydirectedabuse  BiHNet_single
1   0.820046       cad  identitydirectedabuse         single
        auc task_name          task_detail            alg
8  0.877510       cad  persondirectedabuse  BiHNet_single
2  0.863839       cad  persondirectedabuse         single
        auc task_name task_detail            alg
0  0.830102     dygen        hate  BiHNet_single
0  0.851989     dygen        hate         single
        au

In [23]:
all_instant_df.groupby("alg")[metrics_type].mean()

alg
BiHNet_single    0.870197
single           0.884618
Name: auc, dtype: float64

# LATEX 

In [29]:
latex_table = all_instant_df.sort_values(by=["task_name", "task_detail", "alg"]).to_latex(index=False)
new_string = latex_table.replace("BART-BiHNet+Vanilla \\\\", "BART-BiHNet+Vanilla\\\\ \n \hline")
# new_string =  new_string.replace("_", "--")

In [30]:
print(new_string)

\begin{tabular}{rlll}
\toprule
auc & task_name & task_detail & alg \\
\midrule
0.975141 & abusive & abusive & BART-Adapter-Vanilla \\
0.975408 & abusive & abusive & BART-BiHNet+EWC \\
0.972513 & abusive & abusive & BART-BiHNet+Reg \\
0.974635 & abusive & abusive & BART-BiHNet+Vanilla\\ 
 \hline
0.858683 & abusive & hateful & BART-Adapter-Vanilla \\
0.849809 & abusive & hateful & BART-BiHNet+EWC \\
0.864749 & abusive & hateful & BART-BiHNet+Reg \\
0.854976 & abusive & hateful & BART-BiHNet+Vanilla\\ 
 \hline
0.887943 & cad & affiliationdirectedabuse & BART-Adapter-Vanilla \\
0.908008 & cad & affiliationdirectedabuse & BART-BiHNet+EWC \\
0.879390 & cad & affiliationdirectedabuse & BART-BiHNet+Reg \\
0.888610 & cad & affiliationdirectedabuse & BART-BiHNet+Vanilla\\ 
 \hline
0.780956 & cad & identitydirectedabuse & BART-Adapter-Vanilla \\
0.802461 & cad & identitydirectedabuse & BART-BiHNet+EWC \\
0.799686 & cad & identitydirectedabuse & BART-BiHNet+Reg \\
0.800906 & cad & identitydirected

In [31]:
latex_table = all_final_df.sort_values(by=["task_name", "task_detail", "alg"]).to_latex(index=False)
new_string = latex_table.replace("BART-BiHNet-Multitask \\\\", "BART-BiHNet-Multitask \\\\ \n \hline")
print(new_string)

\begin{tabular}{rlll}
\toprule
auc & task_name & task_detail & alg \\
\midrule
0.974823 & abusive & abusive & BART-Adapter-Multitask \\
0.637408 & abusive & abusive & BART-Adapter-Vanilla \\
0.681150 & abusive & abusive & BART-BiHNet+EWC \\
0.966972 & abusive & abusive & BART-BiHNet+Reg \\
0.784075 & abusive & abusive & BART-BiHNet+Vanilla \\
0.940765 & abusive & abusive & BART-BiHNet-Multitask \\ 
 \hline
0.843342 & abusive & hateful & BART-Adapter-Multitask \\
0.763553 & abusive & hateful & BART-Adapter-Vanilla \\
0.728667 & abusive & hateful & BART-BiHNet+EWC \\
0.666253 & abusive & hateful & BART-BiHNet+Reg \\
0.770081 & abusive & hateful & BART-BiHNet+Vanilla \\
0.774595 & abusive & hateful & BART-BiHNet-Multitask \\ 
 \hline
0.878271 & cad & affiliationdirectedabuse & BART-Adapter-Multitask \\
0.618436 & cad & affiliationdirectedabuse & BART-Adapter-Vanilla \\
0.550398 & cad & affiliationdirectedabuse & BART-BiHNet+EWC \\
0.643099 & cad & affiliationdirectedabuse & BART-BiHNet+Re